In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
import pandas as pd
import string, re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Embedding, Dropout
from keras import metrics
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
train_raw_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
train_df = train_raw_df
# train_df = train_raw_df.sample(frac=0.5)
# train_df = train_df.append(train_raw_df[train_raw_df['target'] == 1])
train_n = len(train_df)
test_n = len(test_df)
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [7]:
train_df.groupby('target').count()

,qid,question_text
target,,
0,1225312,1225312
1,80810,80810


In [8]:
train_df.head(1)
test_df.iloc[594,:][1]

'Has anyone gotten banned on Quora for being under 13?'

In [9]:
lemmatizer = WordNetLemmatizer()
stopset = set(stopwords.words('english'))

In [10]:
def process(txt):
    tokens = []
    for token in wordpunct_tokenize(txt):
        if token.isdigit():
            continue
        if all(char in string.punctuation for char in token):
            continue
        
        token = token.lower()
        token = token.strip()  # Strip whitespace and other punctuations
        token = token.strip('_')  # remove _ if any
        token = token.strip('*')
        if token in stopset:
            continue
        tokens.append(token)
        lemmatizer.lemmatize(token)
    return tokens

In [ ]:
train_df['p_txt'] = train_df['question_text'].apply(process)
test_df['p_txt'] = test_df['question_text'].apply(process)

In [ ]:
train_df.head()

In [ ]:
train_df['p_txtcn'] = train_df['p_txt'].apply(lambda tokens: ' '.join(str(v) for v in tokens))
test_df['p_txtcn'] = test_df['p_txt'].apply(lambda tokens: ' '.join(str(v) for v in tokens))

In [ ]:
no_features = 5000
tfidf_vectorizer = TfidfVectorizer(max_features=no_features)
# tf_vectorizer = CountVectorizer(max_features=no_features)

In [ ]:
def vectorize(txt, vectorizer):
    X = vectorizer.fit_transform(txt)
    feature_names = vectorizer.get_feature_names()
    return X, feature_names, vectorizer

In [ ]:
X_tf, tf_feature_names, vectorizer = vectorize(train_df['p_txtcn'], tfidf_vectorizer)

In [ ]:
no_topics = 25
num_iter = 1

In [ ]:
# lda = LatentDirichletAllocation(n_components=no_topics, max_iter=num_iter, learning_method='online', learning_offset=50.,random_state=9, evaluate_every=100).fit(X_tf)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_features = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            if topic[i] >  1.0:
                top_features.append((feature_names[i], topic[i]))
            else:
                continue        
        print("Topic %d:" % (topic_idx))
        print(", ".join([str(val[0])+": "+"%.2f" % val[1] for val in top_features]))

In [ ]:
# display_topics(lda, tf_feature_names, 5)

In [ ]:
# X_lda = lda.transform(X_tf)
y_clf = train_df['target']
# X_train, X_test, y_train, y_test = train_test_split(X_lda, y_clf, test_size=0.2, random_state=0, stratify=y_clf)
X_train, X_test, y_train, y_test = train_test_split(X_tf, y_clf, test_size=0.2, random_state=0, stratify=y_clf)

In [ ]:
# y_svm = train_df['target']

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# from sklearn.svm import SVC
# clf = SVC(C=10, kernel='linear', random_state=9)
clf = RandomForestClassifier(n_estimators=100, random_state=9)
# clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

clf.fit(X_train, y_train)

In [ ]:
p_pred_y = clf.predict(X_test)
confusion_matrix(y_test, p_pred_y)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, p_pred_y).ravel()

acc = (tp + tn) / (tn + fp + fn + tp)
prec = tp / (tp + fp)
rec = tp / (tp + fn)
f1 = 2 * prec * rec / (prec + rec)

In [ ]:
print("accuracy", acc)
print("precision", prec)
print("recall", rec)
print("f1", f1)
print(tn, fp, fn, tp)

In [ ]:
# X_init_lda_test = lda.transform(vectorizer.transform(test_df['p_txtcn']))
# X_init_lda_test = scaler.transform(X_init_lda_test)

In [ ]:
# y_pred_final = clf.predict(X_init_lda_test)
y_pred_final = clf.predict(vectorizer.transform(test_df['p_txtcn']))

In [ ]:
sub = pd.DataFrame({"qid":test_df["qid"].values})
sub['prediction'] = y_pred_final
sub.to_csv("submission.csv", index=False)

In [ ]:
# X_lda[:30]

In [ ]:
# X_lda_df = pd.DataFrame(X_lda)
# X_lda_df["idxmax"] = X_lda_df.idxmax(axis=1)
# X_lda_df.head(2)

In [ ]:
# y_true = train_df["target"]
# y_lda = X_lda_df["idxmax"]

In [ ]:
# print(confusion_matrix(y_true, y_lda)[:2])

#### Trying Deep Learning with GloVe

In [ ]:
def fit_get_sequences(df, vocab_size):
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(df['p_txt'])
    sequences = tokenizer.texts_to_sequences(df['p_txt'])
    return sequences, tokenizer, vocab_size

In [ ]:
def get_sequences(df, tokenizer):
    return tokenizer.texts_to_sequences(df['p_txt'])

In [ ]:
# vocab_size = 1000
# sequences, tokenizer, vocab_size = fit_get_sequences(train_df, vocab_size)
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))
# avg = sum(map(len, sequences)) / len(sequences)
# std = np.sqrt(sum(map(lambda x: (len(x) - avg) ** 2, sequences)) / len(sequences))
# print("Tokens avg {} and std {}".format(avg, std))

# max_length = 100
# X = pad_sequences(sequences, maxlen=max_length)
# X_init_test = pad_sequences(get_sequences(test_df, tokenizer), maxlen=max_length)

# y = to_categorical(np.asarray(train_df['target']))
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
# y = to_categorical(np.asarray(train_df['target']))

In [ ]:
# print('Shape of data:', X.shape)
# print('Shape of labels:', y.shape)

In [ ]:
def generate_embeddings(vocab_size, word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    embeddings_index = {}
    f = open(EMBEDDING_FILE)
    # In the dataset, each line represents a new word embedding
    # The line starts with the word and the embedding values follow
#     for line in f:
#         values = line.split()
#         word = values[0]
#         embedding = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = embedding 
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in f if o.split(" ")[0] in word_index)
    f.close()

    all_embs = np.stack(embeddings_index.values())
    emb_mean = all_embs.mean()
    emb_std = all_embs.std()
    print(emb_mean, emb_std)
    embedding_dim = 300
    nb_words = min(vocab_size, len(word_index))  # How many words are there actually
    # Create a random matrix with the same mean and std as the embeddings
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_dim))
    # The vectors need to be in the same position as their index.
    # Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on
    # Loop over all words in the word index
    for word, i in word_index.items():
        # If we are above the amount of words we want to use we do nothing
        if i >= vocab_size:
            break
        # Get the embedding vector for the word
        embedding_vector = embeddings_index.get(word)
        # If there is an embedding vector, put it in the embedding matrix
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_dim, embedding_matrix    

In [ ]:
# embedding_dim, embedding_matrix = generate_embeddings(vocab_size, word_index)

In [ ]:
def generate_model(embedding_dim, embedding_matrix, max_length, vocab_size, ifembed=True):
    model = Sequential()
    # model.add(Embedding(vocab_size, 100, input_length=max_length))
    if ifembed:
        model.add(
            Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
        model.add(LSTM(32, return_sequences=True))
    else:
        model.add(LSTM(32, return_sequences=True, input_shape=(max_length)))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=[metrics.mae, metrics.categorical_accuracy])
    return model

In [ ]:
# model = generate_model(embedding_dim, embedding_matrix, max_length, vocab_size)

In [ ]:
# batch_size = 1000
# model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.1)

In [ ]:
# y_svm = train_df['target']
# X_train, X_test, y_train, y_test = train_test_split(X, y_svm, test_size=0.2, random_state=0, stratify=y)

# from sklearn.svm import SVC
# clf = SVC(C=100, kernel='linear', random_state=9)
# clf.fit(X_train, y_train)
# p_pred_y = clf.predict(X_test)

# confusion_matrix(y_test, p_pred_y)

In [ ]:
def c_matrix(y_true, y_pred, num_classes=2):
    cm = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    print(cm)
    cm_np = np.asarray(cm)
    TP = np.diag(cm_np)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP
    TN = []
    for i in range(num_classes):
        temp = np.delete(cm, i, 0)
        temp = np.delete(temp, i, 1)
        TN.append(sum(sum(temp)))
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    acc = (TP + TN) / (TP + FP + TN + FN)
    f1 = 2 * prec * rec / (prec + rec)
    print("precision", prec)
    print("recall", rec)
    print("accuracy", acc)
    print("f1", f1)
    return prec, rec, acc, f1

In [ ]:
# y_pred = model.predict(X_test)

# c_matrix(y_test, y_pred, num_classes=2)

In [ ]:
# y_pred_final = model.predict(X_init_test)

In [ ]:
# y_1 = y_pred_final.argmax(axis=1)

In [ ]:
# sub = pd.DataFrame({"qid":test_df["qid"].values})
# sub['prediction'] = y_1
# sub.to_csv("submission.csv", index=False)